# Tourist Itinerary Planner - Notebook Explicatif

## Introduction

Ce notebook présente le fonctionnement interne du projet "Tourist Itinerary Planner". L'objectif de ce projet est de générer automatiquement des itinéraires touristiques optimisés pour une journée dans une ville donnée. Il combine :

1.  **Génération de Données via LLM:** Utilisation de l'API OpenAI (GPT-4o / GPT-4o-mini) pour collecter des informations sur les points d'intérêt (POI) et des faits intéressants (`city_generator.py`).
2.  **Modélisation par Graphe:** Représentation de la ville et de ses POI à l'aide de la bibliothèque NetworkX (`city_graph.py`).
3.  **Calcul des Temps de Trajet:** Estimation des temps de trajet entre POI en utilisant l'API OpenAI ou la distance Haversine (`distance_api.py`).
4.  **Optimisation par Contraintes:** Utilisation de Google OR-Tools (CP-SAT) pour trouver le meilleur itinéraire possible selon des critères définis (intérêt maximisé, temps de trajet minimisé) en respectant les contraintes (horaires, durée de visite, repas, etc.) (`solver.py`).
5.  **Interface Web (Conceptuelle ici):** Une interface web (HTML/CSS/JS) permet aux utilisateurs finaux d'interagir avec le système (non exécutée directement dans ce notebook mais son interaction avec le backend est expliquée).
6.  **Persistance:** Sauvegarde et chargement des graphes de villes pour éviter la régénération coûteuse des données (`city_graph.py`).

## 1. Configuration et Imports

### Installation des dépendances
Assurez-vous d'avoir installé les bibliothèques nécessaires. Vous pouvez le faire via pip :
```bash
pip install openai "google-ortools>=9.0" networkx python-dotenv matplotlib requests
```
(Note : `requests` n'est pas directement dans votre code mais souvent utile, `matplotlib` est dans `city_graph.py` pour la visualisation optionnelle).

In [ ]:
# ### Imports
import openai
import json
import os
import sys
import math
import datetime
import pickle
import networkx as nx
from dotenv import load_dotenv

In [ ]:
# ### Configuration du Chemin d'Accès (Important !)
# Si ce notebook n'est pas à la racine de votre projet,
# ajoutez le chemin vers les répertoires 'src' et 'data'
# pour que les imports fonctionnent correctement.
# Modifiez le chemin relatif si nécessaire.
project_root = os.path.abspath(os.path.join(os.getcwd(), '.')) # Ajustez '../' par '.' si le notebook est à la racine
src_path = os.path.join(project_root, 'src')
data_path = os.path.join(project_root, 'data')

if src_path not in sys.path:
    sys.path.append(src_path)
if data_path not in sys.path:
    sys.path.append(data_path)
if project_root not in sys.path:
     sys.path.append(project_root)

# Importer les modules de votre projet
try:
    from city_generator import generate_city_data, generate_city_fun_facts
    from city_graph import create_graph, save_graph, load_graph, check_city_graph_exists # display_graph_window (optionnel)
    from distance_api import DistanceCalculator
    from solver import TouristItinerarySolver
    print("Modules du projet importés avec succès.")
except ImportError as e:
    print(f"Erreur lors de l'importation des modules: {e}")
    print("Vérifiez que le notebook est dans le bon répertoire ou ajustez `project_root`.")

### Clé API OpenAI
Le système nécessite une clé API OpenAI pour générer les données et calculer les temps de trajet (si l'option API est choisie).

1. Créez un fichier nommé `.env` à la racine de votre projet (ou dans le même dossier que ce notebook).
2. Ajoutez votre clé API dans ce fichier comme ceci :
   ```
   OPENAI_API_KEY=votre_clé_api_commence_par_sk-...
   ```
3. La cellule suivante chargera cette clé.

In [ ]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("Clé API OpenAI chargée avec succès depuis le fichier .env.")
    openai.api_key = api_key
else:
    print("ATTENTION: Clé API OpenAI non trouvée. La génération de données et le calcul de distance via API échoueront.")
    print("Veuillez créer un fichier .env comme décrit ci-dessus.")

## 2. Génération des Données de la Ville (`city_generator.py`)

Le module `city_generator.py` interagit avec l'API OpenAI pour obtenir :
1.  La liste des Points d'Intérêt (POI) : Attractions touristiques et restaurants avec détails (coordonnées, horaires, intérêt, durée, coût...).
2.  Des faits intéressants ("Fun Facts") sur la ville.

### 2.1 Génération des Fun Facts

La fonction `generate_city_fun_facts` utilise un modèle plus léger (gpt-4o-mini) pour obtenir des faits concis.

In [ ]:
city_name_example = "Paris"
run_fun_fact_generation = False # Mettre à True pour exécuter (coût API)

if run_fun_fact_generation and api_key:
    print(f"Génération des fun facts pour {city_name_example}...")
    fun_facts = generate_city_fun_facts(city_name_example, api_key=api_key, count=5)
    print("\nFaits Intéressants Générés:")
    if fun_facts:
        for i, fact in enumerate(fun_facts):
            print(f"- {fact}")
    else:
        print("Aucun fait intéressant n'a pu être généré.")
else:
    print("Génération des fun facts désactivée ou clé API manquante.")
    print("Exemple de sortie attendue:")
    print("- La Tour Eiffel devait être démontée après 20 ans.")
    print("- Le Louvre est le plus grand musée d'art du monde.")

### 2.2 Génération des Données des POI

La fonction `generate_city_data` est plus complexe. Elle utilise GPT-4o pour générer une liste structurée de 80 attractions et 20 restaurants. Elle inclut une étape de "sanitization" pour s'assurer que les types de données (int, float) sont corrects.
**Attention:** Cette opération peut être coûteuse en tokens API et prendre du temps.

In [ ]:
# Vérifions d'abord si un graphe existe déjà pour éviter la génération
city_to_process = "paris" # ou une autre ville de votre choix
graph_exists = False
try:
    # Utilise la fonction de city_graph.py
    # Enveloppé dans try/except car le module data_path pourrait ne pas être défini si l'import échoue
    graph_exists = check_city_graph_exists(city_to_process)
except NameError:
    print("Fonction check_city_graph_exists non trouvée, impossible de vérifier l'existence du graphe.")

print(f"\nUn graphe pour '{city_to_process}' existe-t-il déjà ? {'Oui' if graph_exists else 'Non'}")

# --- Flag pour contrôler l'exécution de la génération ---
# Mettez à True UNIQUEMENT si le graphe n'existe pas ET vous voulez le générer (coûts API !)
run_poi_generation = False and not graph_exists

city_graph = None

if run_poi_generation and api_key:
    print(f"\nATTENTION: Lancement de la génération des données POI pour {city_to_process.capitalize()}...")
    print("Cela peut prendre plusieurs minutes et utiliser des crédits API.")
    try:
        # generate_city_data crée et sauvegarde directement le graphe
        city_graph = generate_city_data(city_to_process, api_key=api_key)
        if city_graph:
            print(f"Données générées et graphe créé pour {city_to_process.capitalize()}.")
            # Afficher quelques infos sur le premier POI généré
            if list(city_graph.nodes):
                first_poi_id = list(city_graph.nodes)[0]
                print("\nExemple de données pour le premier POI généré:")
                # Utilisation de json.dumps pour un affichage propre des dictionnaires
                print(json.dumps(city_graph.nodes[first_poi_id], indent=2, ensure_ascii=False))
        else:
            print(f"Échec de la génération des données pour {city_to_process.capitalize()}.")
    except NameError:
         print("Fonction generate_city_data non trouvée. Vérifiez les imports.")
    except Exception as e:
         print(f"Une erreur est survenue pendant la génération: {e}")
elif graph_exists:
    print(f"\nChargement du graphe existant pour {city_to_process.capitalize()}...")
    try:
        city_graph = load_graph(city_to_process)
        if city_graph:
            print(f"Graphe chargé avec succès ({len(city_graph.nodes)} noeuds, {len(city_graph.edges)} arêtes).")
            # Afficher un exemple de POI du graphe chargé
            if list(city_graph.nodes):
                # Prend un ID au hasard ou le premier
                sample_poi_id = list(city_graph.nodes)[0]
                print("\nExemple de données pour un POI du graphe chargé:")
                print(json.dumps(city_graph.nodes[sample_poi_id], indent=2, ensure_ascii=False))
        else:
            print("Échec du chargement du graphe existant.")
    except NameError:
         print("Fonction load_graph non trouvée. Vérifiez les imports.")
    except Exception as e:
         print(f"Une erreur est survenue pendant le chargement: {e}")

else:
    print("\nGénération des POI désactivée ou clé API manquante, et aucun graphe existant trouvé.")
    print("Pour la suite de la démonstration, certaines étapes nécessiteront un graphe.")
    print("Exemple de structure attendue pour un POI:")
    print("""
    {
      "ID": 1,
      "Nom": "Tour Eiffel",
      "Horaire": "09:00-23:45",
      "Type": "Touristique",
      "Interet": 10,
      "duree": 90,
      "cout": 28.30,
      "latitude": 48.8584,
      "longitude": 2.2945
    }""")

## 3. Représentation par Graphe (`city_graph.py`)

Le module `city_graph.py` utilise `NetworkX` pour modéliser la ville :
- Chaque **nœud** est un POI (ID entier) avec ses attributs (nom, type, coords, etc.).
- Les **arêtes** connectent initialement tous les POI. Elles stockeront ensuite les temps de trajet calculés.
- Les fonctions `save_graph` et `load_graph` utilisent `pickle` pour la persistance.

(Le chargement/sauvegarde a été démontré dans la section précédente lors de l'appel à `generate_city_data` ou `load_graph`)

### Visualisation (Optionnelle)
La fonction `display_graph_window` peut afficher le graphe (nécessite matplotlib).
Attention: Peut être lent pour un grand nombre de POIs.

In [ ]:
display_the_graph = False and city_graph is not None # Mettre à True pour essayer d'afficher

if display_the_graph:
    print("\nTentative d'affichage du graphe (peut prendre du temps)...")
    try:
        # Assurez-vous que matplotlib est installé et fonctionne dans votre environnement Jupyter
        from city_graph import display_graph_window
        # %matplotlib inline # Décommentez et choisissez inline ou qt si nécessaire
        display_graph_window(city_graph, city=city_to_process.capitalize())
        print("Si aucune fenêtre ne s'affiche, vérifiez votre backend matplotlib (%matplotlib ...)")
    except ImportError:
        print("La fonction display_graph_window ou matplotlib n'est pas disponible.")
    except Exception as e:
        print(f"Erreur lors de l'affichage du graphe: {e}")
else:
    print("\nVisualisation du graphe désactivée ou graphe non chargé.")

## 4. Calcul des Temps de Trajet (`distance_api.py`)

La classe `DistanceCalculator` gère les estimations de temps de trajet.
- Elle peut utiliser l'API OpenAI (plus précis, coûteux) ou une formule Haversine (rapide, approximatif).
- Elle implémente un cache en mémoire pour éviter les appels redondants.
- Elle supporte le traitement par lots (batching) pour optimiser les appels API.

### Instanciation
On peut choisir d'utiliser l'API ou non lors de l'instanciation.

In [ ]:
use_api_for_distance_demo = False # Mettre à True pour utiliser l'API (coûts !)
distance_calculator = None

try:
    if api_key:
        distance_calculator = DistanceCalculator(api_key=api_key, use_api=use_api_for_distance_demo)
        print(f"DistanceCalculator instancié (mode API: {'Activé' if use_api_for_distance_demo else 'Désactivé - Haversine'}).")
    else:
        # Fournir une clé factice si aucune clé n'est chargée, pour éviter une erreur à l'instanciation
        # Le mode 'use_api=False' garantit qu'elle ne sera pas utilisée.
        distance_calculator = DistanceCalculator(api_key="dummy_key_not_used", use_api=False)
        print("DistanceCalculator instancié (mode API: Désactivé - Clé API manquante). Utilisation de Haversine.")
except NameError:
    print("Classe DistanceCalculator non trouvée. Vérifiez les imports.")

### Exemple de Calcul
Utilisons deux POIs (fictifs ou réels si le graphe est chargé) pour tester.

In [ ]:
poi_1 = None
poi_2 = None

if city_graph and len(city_graph.nodes) >= 2:
    nodes_list = list(city_graph.nodes)
    poi_1_id = nodes_list[0]
    poi_2_id = nodes_list[1]
    # Assurez-vous que les données du nœud sont bien accessibles
    if poi_1_id in city_graph.nodes and poi_2_id in city_graph.nodes:
        poi_1 = city_graph.nodes[poi_1_id]
        poi_2 = city_graph.nodes[poi_2_id]
        # Vérification minimale que les POIs ont les clés nécessaires
        if 'Nom' in poi_1 and 'latitude' in poi_1 and 'longitude' in poi_1 and \
           'Nom' in poi_2 and 'latitude' in poi_2 and 'longitude' in poi_2:
             print(f"\nUtilisation de POI réels du graphe : {poi_1.get('Nom', poi_1_id)} et {poi_2.get('Nom', poi_2_id)}")
        else:
             print("\nLes POIs chargés du graphe n'ont pas les attributs nécessaires (Nom, latitude, longitude).")
             poi_1 = None # Réinitialiser pour utiliser les fictifs
             poi_2 = None
    else:
        print(f"\nLes IDs de POI {poi_1_id} ou {poi_2_id} n'ont pas été trouvés dans les nœuds du graphe.")

# POIs fictifs si pas de graphe ou si les POIs réels sont invalides
if poi_1 is None or poi_2 is None:
    poi_1 = {"ID": 101, "Nom": "Point A (Fictif)", "latitude": 48.85, "longitude": 2.35}
    poi_2 = {"ID": 102, "Nom": "Point B (Fictif)", "latitude": 48.86, "longitude": 2.36}
    print("\nUtilisation de POI fictifs.")

if poi_1 and poi_2 and distance_calculator:
    try:
        # Calcul pour la marche (mode=0)
        mode_marche = 0 # 0: marche, 1: transports en commun, 2: voiture
        travel_time_walk = distance_calculator.get_travel_time(poi_1, poi_2, mode_marche)
        print(f"Temps de trajet estimé (marche) entre '{poi_1['Nom']}' et '{poi_2['Nom']}': {travel_time_walk} minutes.")

        # Calcul pour les transports en commun (mode=1)
        mode_transport = 1
        travel_time_transit = distance_calculator.get_travel_time(poi_1, poi_2, mode_transport)
        print(f"Temps de trajet estimé (transports) entre '{poi_1['Nom']}' et '{poi_2['Nom']}': {travel_time_transit} minutes.")

        # Important : vider la file d'attente si le batching est utilisé et qu'on veut les résultats immédiatement
        distance_calculator.flush_queue()
        print(f"Nombre total de requêtes API effectuées par ce calculateur: {distance_calculator.get_request_count()}")
    except Exception as e:
        print(f"Une erreur est survenue lors du calcul du temps de trajet: {e}")
elif not distance_calculator:
    print("\nLe calculateur de distance n'a pas pu être instancié.")

## 5. Optimisation de l'Itinéraire (`solver.py`)

Le module `solver.py` contient la classe `TouristItinerarySolver`, qui est le cœur de l'optimisation.

- **Initialisation:** Prend en paramètres la ville, les heures, les contraintes (visites obligatoires, restaurants), la clé API, etc. Charge le graphe correspondant.
- **Pré-calculs:**
    - Calcule les `k` plus proches voisins pour chaque POI (basé sur Haversine) pour limiter la complexité.
    - Détermine et met en cache le mode de transport "préféré" et le temps de trajet associé entre les voisins et POIs obligatoires. Cela utilise `DistanceCalculator` (API ou Haversine selon le choix). *Cette étape peut nécessiter des appels API si `use_api_for_distance` est True.*
- **Modélisation CP-SAT:** Construit un modèle de Programmation par Contraintes avec OR-Tools :
    - **Variables:** Visites (bool), positions (bool), heures d'arrivée/départ (int).
    - **Contraintes:** Unicité, séquencement, horaires d'ouverture, durée de visite, temps de trajet, contraintes de repas, visites obligatoires, nombre max de POI.
    - **Objectif:** Maximiser `(Score d'Intérêt Total * 10) - Temps de Trajet Total`.
- **Résolution:** Appelle le solveur CP-SAT pour trouver une solution optimale ou faisable.
- **Formatage:** Met en forme la solution trouvée en un itinéraire textuel lisible.

### Instanciation et Résolution (Exemple)

Définissons les paramètres pour une journée à Paris.
**Important:** Cette étape fonctionnera mieux si un graphe pour `city_to_process` a été chargé ou généré précédemment.

In [ ]:
params = {
    "city": city_to_process,
    "start_time": "09:00",
    "end_time": "18:00",
    "max_pois": 6, # Nombre maximum de lieux à visiter (restaurants inclus)
    "restaurant_count": 1, # Nombre de restaurants souhaités (0, 1 ou 2)
    "mandatory_visits": [], # Liste d'IDs de POI obligatoires (ex: [1, 5] si le graphe est chargé)
    "use_api_for_distance": False, # False pour utiliser Haversine (plus rapide/moins cher pour la démo)
    "api_key": api_key
}

# Vérifier si on peut lancer le solveur
# On a besoin d'une instance de graphe valide et potentiellement d'une clé API (si use_api_for_distance est True)
can_run_solver = city_graph is not None
if params["use_api_for_distance"] and not api_key:
    can_run_solver = False
    print("\nClé API requise mais non disponible pour utiliser l'API de distance.")

solver_instance = None
itinerary_solution = None

if can_run_solver:
    print("\nInstanciation du solveur avec les paramètres suivants:")
    # Affichage propre des paramètres
    print(json.dumps(params, indent=2))

    # Instanciation (peut prendre du temps pour les pré-calculs si API activée)
    try:
        solver_instance = TouristItinerarySolver(
            city=params["city"],
            graph=city_graph, # Fournir le graphe chargé/généré
            start_time=params["start_time"],
            end_time=params["end_time"],
            max_pois=params["max_pois"],
            restaurant_count=params["restaurant_count"],
            mandatory_visits=params["mandatory_visits"],
            api_key=params["api_key"],
            use_api_for_distance=params["use_api_for_distance"]
        )
        print("Solveur instancié avec succès.")

        # Résolution (peut prendre jusqu'à 60 secondes par défaut)
        print("\nLancement de la résolution de l'itinéraire...")
        itinerary_solution = solver_instance.solve() # Utilise max_pois de l'instance

        if itinerary_solution:
            print("\nItinéraire trouvé !")
            # Formatage et affichage
            formatted_itinerary = solver_instance.format_itinerary(itinerary_solution)
            print("\n--- ITINÉRAIRE OPTIMISÉ ---")
            print(formatted_itinerary)
            print("---------------------------\n")

            # On peut aussi récupérer le dictionnaire pour l'API web
            # try:
            #     itinerary_dict = solver_instance._convert_itinerary_to_dict(itinerary_solution)
            #     print("\nFormat Dictionnaire (pour API):")
            #     print(json.dumps(itinerary_dict, indent=2, ensure_ascii=False))
            # except AttributeError:
            #     print("La méthode _convert_itinerary_to_dict n'existe pas dans cette version du solveur.")

        else:
            print("\nAucun itinéraire faisable n'a été trouvé avec les contraintes données.")

    except NameError:
        print("Classe TouristItinerarySolver non trouvée. Vérifiez les imports.")
    except Exception as e:
        import traceback
        print(f"\nErreur lors de l'instanciation ou de la résolution: {e}")
        print("Traceback:")
        traceback.print_exc()

else:
    print("\nImpossible de lancer le solveur : graphe non chargé ou conditions non remplies (e.g., API requise mais clé manquante).")

## 6. Intégration avec l'Interface Web (Conceptuel)

Ce notebook se concentre sur le backend Python. L'interface web fournie (`index.html`, `style.css`, `script.js`) interagit avec ce backend via des appels API HTTP. Typiquement :

1.  **Serveur Backend:** Un framework web Python (comme Flask, FastAPI ou Django) est nécessaire pour exposer des endpoints API. Il n'est pas inclus dans les fichiers fournis mais est implicitement requis.
2.  **Endpoint `/api/plan` (POST):**
    *   Le frontend envoie les paramètres du formulaire (ville, heures, contraintes, `use_api_for_distance`) en JSON.
    *   Le backend reçoit la requête, instancie `TouristItinerarySolver` avec ces paramètres (chargeant ou générant le graphe si besoin), appelle `solver.solve()`.
    *   Il formate le résultat (succès ou erreur) en JSON et le renvoie au frontend. La fonction `_convert_itinerary_to_dict` ou `format_itinerary` est utilisée ici.
3.  **Endpoint `/api/fun-facts` (POST):**
    *   Le frontend envoie le nom de la ville en JSON.
    *   Le backend appelle `generate_city_fun_facts`.
    *   Il renvoie la liste des faits (ou un message d'erreur) en JSON.
4.  **Frontend (JavaScript):**
    *   `script.js` utilise la `fetch` API pour envoyer les requêtes aux endpoints ci-dessus.
    *   Il gère l'affichage de l'état de chargement (spinner, faits intéressants).
    *   Il met à jour le DOM (la page HTML) avec l'itinéraire reçu ou les messages d'erreur.

## 7. Conclusion et Perspectives

Ce notebook a détaillé les étapes clés du projet "Tourist Itinerary Planner", depuis la génération des données jusqu'à l'optimisation de l'itinéraire. Le système combine des technologies modernes (LLM, CP-SAT) pour résoudre un problème de planification complexe.

**Points Forts:**
- Génération automatique des données de base.
- Optimisation puissante via OR-Tools.
- Prise en compte de nombreuses contraintes réalistes (horaires, repas, durée).
- Flexibilité dans le calcul des temps de trajet (API vs Haversine).
- Persistance des données via les graphes sauvegardés.
- (Conceptuellement) Accessible via une interface web.

**Perspectives d'Amélioration (Rappel):**
- Intégration d'API cartographiques pour des données/temps de trajet plus fiables/temps réel.
- Amélioration de l'interface web (carte interactive, sélection de POI assistée).
- Personnalisation plus fine (profils d'intérêt, budgets).
- Planification multi-jours.
- Feedback utilisateur pour affiner les scores/heuristiques.

--- Fin du Notebook ---